In [7]:
import pickle
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset

# Load the data
with open('datasets/X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open('datasets/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open('datasets/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('datasets/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

In [9]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

/home/vizuamaix/Downloads/4th year/My Research Codes/Project/jupyter/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   1%|1         | 10.5M/714M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [8]:
# Convert the Series to a list of strings
X_train_texts = X_train['sinhala_sentence'].tolist()
X_test_texts = X_test['sinhala_sentence'].tolist()

X_train_texts

['විශේෂ ෆ්\u200dරඩ්   රයිස්  එකනම් සුපිරි',
 'සේවාව හොඳයි',
 'මට්ටන් සූප් එක නියමයි , ගානත් ශේප් ',
 'ආ මේ යනවා කිව්ව ට්\u200dරිප්  එකද?',
 'රිලැක්ස් කරන්න  හොඳ තැනක්, බොන්න ගොඩක් තිබුණා',
 'ටිකක් ගනන් වැඩියි  ඒ උනාට  කෑමනම් හොඳයි සේවාව ඒකෙත් අඩුවක් නෑහැ ',
 'එබිලිබිටියේ බන් ඔය ඉන්නේ ',
 'හරිම ලස්සනයි ඩියර් ',
 'කෑම හොඳයි ,ගාන සාමාන්\u200dයය',
 'ලොකු පෝශන් හා සේවාව හොඳයි',
 'පොන්නයොත් නිලමේට අඳිනවා එකයි  ',
 'වෙන්න හබි ජපන් නේ ',
 'මම කිව්වොත් මේක මම පටිගත කිරීම කළේ නැහැ කියලා?',
 'දියණිය, අපි ගීතයක් ගායනා කරමු ද? ඔබේ පවුලේ අය සමඟ විනෝද වන්න',
 'සේවාව ගාස්තු එකක් එකතු  කරන තරමට සේවාවක් නෑහැ ',
 'මුහුදු ආහාර රයිස් එකේචික්කන් තමයි වැඩියෙන් තිබ්බේ ',
 'ඔව් ඔව් මම තමා ',
 'නිකම්ම මෙම ස්ථානය හම්බුනා,සුපිරි කෑම',
 'දුන්නේ නෑ නේද ',
 'මට්ටන් කරි  එකට හිත  ගියා සුපිරි',
 'ස්තූතියි මල්ලි ',
 'ටීවී බලන එක සහ මුලින්ම නතර කරලා තියෙන්නේ බලන්න දෙයක් නැති  නිසාම ',
 'කෑම රහ නෑ....කාර්ය මණ්ඩලය හරිම   අකාරුණිකයි,සේවාව මාරම මන්දගාමී',
 'අවුලක් නෑ ',
 'එළවලු කොත්තු හා ධාල්  කරි  ගොඩාක් රසයි හා සේවාව ගොඩා

In [12]:
# Tokenize the inputs
train_encodings = tokenizer(X_train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test_texts, truncation=True, padding=True, max_length=128)

In [17]:
import pandas as pd

# Combine the labels from training and test sets
all_labels = pd.concat([y_train, y_test])

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the LabelEncoder on the combined labels
label_encoder.fit(all_labels)


LabelEncoder()

In [18]:
# Transform the labels separately
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert these encoded labels to PyTorch tensors
train_labels = torch.tensor(y_train_encoded)
test_labels = torch.tensor(y_test_encoded)


In [19]:
from torch.utils.data import TensorDataset

# Create TensorDatasets for training and test sets
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']), 
                              torch.tensor(train_encodings['attention_mask']), 
                              train_labels)

test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']), 
                             torch.tensor(test_encodings['attention_mask']), 
                             test_labels)


In [20]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Define DataLoaders
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=8)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=8)


In [21]:
from transformers import AdamW

# Set up the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)


/home/vizuamaix/Downloads/4th year/My Research Codes/Project/jupyter/venv/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
# Check unique values in y_train_encoded
print(f"Unique values in y_train_encoded: {torch.unique(train_labels)}")
print(f"Unique values in y_test_encoded: {torch.unique(test_labels)}")


Unique values in y_train_encoded: tensor([  0,   2,   3,   4,   5,  11,  12,  13,  15,  16,  19,  20,  21,  23,
         24,  25,  26,  27,  32,  34,  37,  39,  40,  42,  46,  52,  53,  55,
         56,  57,  59,  60,  62,  64,  69,  70,  76,  77,  78,  80,  81,  83,
         85,  87,  88,  90,  94,  95,  96,  97,  98, 100, 106, 107, 112, 119,
        121, 123, 128, 129, 130, 131, 132, 133, 135, 137, 139, 141, 142, 143,
        145, 146, 151, 152, 153, 154, 156, 157, 158, 162, 163, 164, 167, 168,
        169, 171, 173, 175, 178, 179, 188, 189, 190, 191, 194, 197, 200, 201,
        202, 204, 208, 209, 211, 213, 214, 215, 217, 218, 224, 225, 226, 229,
        231, 232, 233, 235, 237, 238, 241, 243, 244, 245, 246, 247, 249, 250,
        251, 255, 257, 258, 262, 265, 267, 268, 269, 270, 272, 273, 275, 276,
        277, 283, 284, 285, 287, 288, 291, 292, 293, 295, 296, 297, 304])
Unique values in y_test_encoded: tensor([  1,   6,   7,   8,   9,  10,  14,  17,  18,  22,  28,  29,  30,  31,
 

In [26]:
import torch
import numpy as np  # Import numpy

# Convert numpy array to PyTorch tensor if necessary
if isinstance(y_train_encoded, np.ndarray):
    y_train_encoded = torch.tensor(y_train_encoded)

# Calculate the number of unique labels
num_labels = len(torch.unique(y_train_encoded))
print(f"Number of unique labels: {num_labels}")


Number of unique labels: 153


In [27]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=153)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [28]:
# Ensure all labels are within the valid range
max_label = torch.max(torch.cat((y_train_encoded, y_test_encoded)))
print(f"Maximum label value: {max_label}")

if max_label >= 153:
    print("Some labels are out of bounds.")
else:
    print("All labels are within the valid range.")


TypeError: expected Tensor as element 1 in argument 0, but got numpy.ndarray